In [1]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.cross_decomposition import PLSRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
from sklearn.utils import resample
from scipy.stats import pearsonr, norm
from pingouin import partial_corr
import matplotlib.lines as mlines
warnings.filterwarnings("ignore")

In [3]:
def fisher_z(r):
    return 0.5 * np.log((1 + r) / (1 - r))
def reverse_fisher_z(z):
    return (np.exp(2*z) - 1) / (np.exp(2*z) + 1)

def load_ratings_data_with_movies(va_dir, category_dir, subject, brain_region, data_type, is_yhat=True):
    if data_type == 'valence_arousal' or data_type == 'binary_valence_arousal':
        dir_path = va_dir
    elif data_type == 'category':
        dir_path = category_dir
    else:
        raise ValueError("Invalid data type. Choose 'valence_arousal' or 'category'.")
    
    file_prefix = 'yhat' if is_yhat else 'y'
    file_path = os.path.join(dir_path, f'{file_prefix}_sub-{subject}_{brain_region}.csv')
    if not os.path.exists(file_path) and brain_region == 'vmPFC_a24_included':
        file_path = os.path.join(dir_path, f'{file_prefix}_sub-{subject}_vmPFC.csv')
    data = pd.read_csv(file_path)
    return data

def load_TEM_data_with_movies(TEM_dir, subject, brain_region, component, freq_name, is_yhat=True):
    file_prefix = 'yhat' if is_yhat else 'y'
    file_path = os.path.join(TEM_dir, f'{file_prefix}_sub-{subject}_{brain_region}_{component}_{freq_name}.csv')
    if not os.path.exists(file_path) and brain_region == 'vmPFC_a24_included':
        file_path = os.path.join(TEM_dir, f'{file_prefix}_sub-{subject}_vmPFC_{component}_{freq_name}.csv')
    data = pd.read_csv(file_path)

    return data

def cross_decoding_TEMRatings(data, emo_labels, if_output_cross_decoding_yhat=False):

    yhat_TEM, y_rating = data
    pred_outcome_corr = {}
    cross_decoding_yhat = pd.DataFrame()

    X = yhat_TEM.copy()
    X = X.drop(columns=['movie']).values
    y = y_rating[emo_labels].values  
    target_labels = emo_labels

    #make sure order of movies movie column is matched
    assert (yhat_TEM['movie'] == y_rating['movie']).all(), 'Movie order is not matched'
        
    logo = LeaveOneGroupOut()

    # Perform cross-validation by movie group
    for train_idx, test_idx in logo.split(X, y, groups=y_rating['movie']):
        # Perform PLS regression
        model = PLSRegression(n_components=20)
        model.fit(X[train_idx], y[train_idx])
        predictions = model.predict(X[test_idx])
        predictions_wlabels = {}
        # Calculate correlation for each target label 
        for label in target_labels:
            corr = np.corrcoef(predictions[:, target_labels.index(label)], y[test_idx][:, target_labels.index(label)])[0, 1]
            if label not in pred_outcome_corr:
                pred_outcome_corr[label] = []
            pred_outcome_corr[label].append(corr)
            if if_output_cross_decoding_yhat:
                predictions_wlabels[label] = predictions[:, target_labels.index(label)]
        predictions_wlabels=pd.DataFrame(predictions_wlabels)
        #add current group/movie names to the predictions_wlabels df
        predictions_wlabels['movie'] = y_rating['movie'].values[test_idx]
        cross_decoding_yhat = pd.concat([cross_decoding_yhat, predictions_wlabels], ignore_index=True)
    
    # Calculate the mean correlation for each target label across cross-validation iterations (movies)
    mean_corr = {label: np.mean(pred_outcome_corr[label]) for label in target_labels}
    if if_output_cross_decoding_yhat:
        return mean_corr, cross_decoding_yhat
    else:
        return mean_corr

def mediation_analysis(yhat_BrainToRatings, yhat_BrainToTEM, y_ratings, labels, n_bootstraps=1000):
    results = []

    for label in labels:
        X = sm.add_constant(yhat_BrainToRatings[label])  # Add intercept

        #yhat_BrainToTEM ~ yhat_BrainToRatings
        mediator_model = sm.OLS(yhat_BrainToTEM[label], X).fit()
        a = mediator_model.params[1]  # Effect of predictor on mediator

        # y_ratings ~ yhat_BrainToRatings + yhat_BrainToTEM
        X2 = sm.add_constant(pd.concat([yhat_BrainToRatings[label], yhat_BrainToTEM[label]], axis=1))
        outcome_model = sm.OLS(y_ratings[label], X2).fit()
        b = outcome_model.params[2]  # effect of mediator on outcome (controlling for predictor)
        c_prime = outcome_model.params[1]  # direct effect of predictor on outcome

        indirect_effect = a * b
        total_effect = indirect_effect + c_prime  

        # Bootstrapping for significance testing
        boot_indirect = []
        for _ in range(n_bootstraps):
            idx = resample(range(len(yhat_BrainToRatings)), replace=True)
            X_boot = sm.add_constant(yhat_BrainToRatings.iloc[idx][label])
            mediator_model_boot = sm.OLS(yhat_BrainToTEM.iloc[idx][label], X_boot).fit()
            a_boot = mediator_model_boot.params[1]

            X2_boot = sm.add_constant(pd.concat([yhat_BrainToRatings.iloc[idx][label], yhat_BrainToTEM.iloc[idx][label]], axis=1))
            outcome_model_boot = sm.OLS(y_ratings.iloc[idx][label], X2_boot).fit()
            b_boot = outcome_model_boot.params[2]

            boot_indirect.append(a_boot * b_boot)

        boot_indirect = np.array(boot_indirect)
        p_value = (np.sum(boot_indirect <= 0) + np.sum(boot_indirect >= 0)) / n_bootstraps  # Two-tailed test

        results.append({
            "Category": label,
            "a (Predictor → Mediator)": a,
            "b (Mediator → Outcome)": b,
            "c' (Direct Effect)": c_prime,
            "Indirect Effect (a*b)": indirect_effect,
            "Total Effect": total_effect,
            "p-value (bootstrapped)": p_value
        })

    return pd.DataFrame(results)

def fisher_r_to_z(r):
    return 0.5 * np.log((1 + r) / (1 - r))

def get_partial_corrs(yhat_BrainToRatings, yhat_BrainToTEM, y_ratings, labels):
    results = []

    for label in labels:
        results_r_xy_movie = []
        results_r_xy_given_m_movie = []
        for movie in yhat_BrainToRatings['movie'].unique():
            yhat_BrainToRatings_movie = yhat_BrainToRatings[yhat_BrainToRatings['movie'] == movie][label]
            yhat_BrainToTEM_movie = yhat_BrainToTEM[yhat_BrainToTEM['movie'] == movie][label]
            y_ratings_movie = y_ratings[y_ratings['movie'] == movie][label]
            # Calculate Pearson correlation
            r_xy, p_xy = pearsonr(yhat_BrainToRatings_movie, y_ratings_movie)
            # Partial correlation controlling for mediator
            df = pd.DataFrame({
                "X": yhat_BrainToRatings_movie, 
                "M": yhat_BrainToTEM_movie, 
                "Y": y_ratings_movie
            })
            partial_corr_result = partial_corr(data=df, x="X", y="Y", covar="M", method="pearson")
            r_xy_given_m = partial_corr_result["r"].values[0]  # Partial correlation
            results_r_xy_movie.append(r_xy)
            results_r_xy_given_m_movie.append(r_xy_given_m)
        #average across movies
        r_xy = np.mean(results_r_xy_movie)
        r_xy_given_m = np.mean(results_r_xy_given_m_movie)
        # Convert to Fisher's z
        r_xy_z = fisher_r_to_z(r_xy)
        r_xy_given_m_z = fisher_r_to_z(r_xy_given_m)
  
        results.append({
            "Emotion": label,
            "r_xy": r_xy,
            "r_xy_given_m": r_xy_given_m,
            "r_xy_z": r_xy_z,
            "r_xy_given_m_z": r_xy_given_m_z
        })
    return pd.DataFrame(results)          


def partial_correlation_BrainTEMRatings(TEM_dir, va_dir, category_dir, subjects, categories, va_ratings, emo_types=['category', 'valence_arousal','valence_arousal'], brain_regions=['Hippocampus', 'EntorhinalCortex','vmPFC_a24_included'], components=['p','g','g'], freq_names=['freq01234','freq01234','freq01234']):
    for subject in tqdm(subjects):
        for emo_type, brain_region, component, freq_name in zip(emo_types, brain_regions, components, freq_names):
            labels = categories if emo_type == 'category' else va_ratings
            y_ratings = load_ratings_data_with_movies(va_dir, category_dir, subject, brain_region, emo_type, is_yhat=False)
            yhat_BrainToRatings = load_ratings_data_with_movies(va_dir, category_dir, subject, brain_region, emo_type, is_yhat=True)
            yhat_BrainToTEM = load_TEM_data_with_movies(TEM_dir, subject, brain_region, component, freq_name, is_yhat=True)
            # Perform cross-decoding
            _, yhat_BrainToTEMToRating = cross_decoding_TEMRatings([yhat_BrainToTEM, y_ratings], labels, if_output_cross_decoding_yhat=True)
            #mediation_results = mediation_analysis(yhat_BrainToRatings, yhat_BrainToTEMToRating, y_ratings, labels)
            partial_corr_results = get_partial_corrs(yhat_BrainToRatings, yhat_BrainToTEMToRating, y_ratings, labels)
            partial_corr_results['subject'] = subject
            partial_corr_results['region'] = brain_region
            partial_corr_results['TEM_component'] = component
            partial_corr_results['emo_type'] = emo_type
            partial_corr_results['freq'] = freq_name
            if subject == subjects[0]:
                partial_corr_df = partial_corr_results
            else:
                partial_corr_df = pd.concat([partial_corr_df, partial_corr_results], ignore_index=True)
    return partial_corr_df


def get_TEMRating_cross_decoding_df(TEM_dir, va_dir, category_dir, ori_va_dir, ori_category_dir, subjects, brain_regions, categories, va_ratings, TEM_components):
    pred_outcome_corr_TEMcategory = []
    pred_outcome_corr_TEMva = []

    for emo_type, labels in zip(['category', 'valence_arousal'], [categories, va_ratings]):
        subject_column, region_column, component_column = [], [], []
        for subject in tqdm(subjects):
            for brain_region in brain_regions:
                for component in TEM_components:
                    y_rating = load_ratings_data_with_movies(va_dir, category_dir, subject, brain_region, emo_type, is_yhat=False)
                    yhat_TEM = load_TEM_data_with_movies(TEM_dir, subject, brain_region, component, 'freq01234', is_yhat=True)
                    # Perform cross-decoding 
                    if emo_type == 'category':
                        pred_outcome_corr_TEMcategory.append(cross_decoding_TEMRatings([yhat_TEM, y_rating], labels))
                    else:
                        pred_outcome_corr_TEMva.append(cross_decoding_TEMRatings([yhat_TEM, y_rating], labels))
                    subject_column.append(subject)
                    region_column.append(brain_region)
                    component_column.append(component)
        if emo_type == 'category':
            pred_outcome_corr_TEMcategory = pd.DataFrame(pred_outcome_corr_TEMcategory)
            pred_outcome_corr_TEMcategory['subject'] = subject_column
            pred_outcome_corr_TEMcategory['region'] = region_column
            pred_outcome_corr_TEMcategory['TEMcomponent'] = component_column
        else:
            pred_outcome_corr_TEMva = pd.DataFrame(pred_outcome_corr_TEMva)
            pred_outcome_corr_TEMva['subject'] = subject_column
            pred_outcome_corr_TEMva['region'] = region_column
            pred_outcome_corr_TEMva['TEMcomponent'] = component_column

    pred_outcome_corr_TEMva['target'] = 'valence_arousal'
    pred_outcome_corr_TEMcategory['target'] = 'category'
    #melt each and combine
    pred_outcome_corr_TEMva = pd.melt(pred_outcome_corr_TEMva, id_vars=['subject', 'region', 'target', 'TEMcomponent'], var_name='label', value_name='corr')
    pred_outcome_corr_TEMva['corr_z'] = fisher_z(pred_outcome_corr_TEMva['corr'])
    pred_outcome_corr_TEMcategory = pd.melt(pred_outcome_corr_TEMcategory, id_vars=['subject', 'region', 'target', 'TEMcomponent'], var_name='label', value_name='corr')
    pred_outcome_corr_TEMcategory['corr_z'] = fisher_z(pred_outcome_corr_TEMcategory['corr'])
    pred_outcome_corr = pd.concat([pred_outcome_corr_TEMva, pred_outcome_corr_TEMcategory], ignore_index=True)

    ori_pred_outcome_corr_category = pd.read_csv(ori_category_dir)
    ori_pred_outcome_corr_va = pd.read_csv(ori_va_dir)
    ori_pred_outcome_corr_category = ori_pred_outcome_corr_category[ori_pred_outcome_corr_category['region'].isin(brain_regions)]
    ori_pred_outcome_corr_va = ori_pred_outcome_corr_va[ori_pred_outcome_corr_va['region'].isin(brain_regions)]
    #melt the original data
    ori_pred_outcome_corr_category = pd.melt(ori_pred_outcome_corr_category, id_vars=['subject', 'region'], var_name='label', value_name='corr')
    ori_pred_outcome_corr_category['target'] = 'category'
    ori_pred_outcome_corr_category['TEMcomponent'] = np.nan
    ori_pred_outcome_corr_va = pd.melt(ori_pred_outcome_corr_va, id_vars=['subject', 'region'], var_name='label', value_name='corr')
    ori_pred_outcome_corr_va['target'] = 'valence_arousal'
    ori_pred_outcome_corr_va['TEMcomponent'] = np.nan
    #combine the original data
    ori_pred_outcome_corr = pd.concat([ori_pred_outcome_corr_category, ori_pred_outcome_corr_va], ignore_index=True)
    ori_pred_outcome_corr['decoding_type'] = 'within'
    ori_pred_outcome_corr['corr_z'] = fisher_z(ori_pred_outcome_corr['corr'])
    #combine the original and new data
    pred_outcome_corr['decoding_type'] = 'cross'
    #match the subject format
    pred_outcome_corr['subject'] = ['sub-'+ subj for subj in pred_outcome_corr['subject']]
    all_pred_outcome_corr = pd.concat([ori_pred_outcome_corr, pred_outcome_corr], ignore_index=True)
    return all_pred_outcome_corr

def plot_within_corss_comparison(all_pred_outcome_corr, brain_regions, components):

    plt.figure(figsize=(8*len(brain_regions), 8*len(components)))

    for i, region in enumerate(brain_regions):
        for c, component in enumerate(components):
          
            all_pred_outcome_corr_subset = all_pred_outcome_corr[(all_pred_outcome_corr['region'] == region) & ((all_pred_outcome_corr['TEMcomponent'] == component) | (all_pred_outcome_corr['TEMcomponent'].isna()))]
            all_pred_outcome_corr_subset = all_pred_outcome_corr_subset.drop(columns=['TEMcomponent'])
            plt.subplot(len(components), len(brain_regions), i*len(components)+c+1)
            sns.stripplot(x='label', y='corr', hue='decoding_type', data=all_pred_outcome_corr_subset, jitter=True, dodge=True, alpha=0.7, palette='Set2')
            plt.title(f'TEM {component} yhat to Ratings Cross-Decoding Performance\nin {region}')
            plt.xlabel('')
            plt.ylabel('Prediction-Outcome Correlation')
            if i > 0:
                plt.ylabel('')
            plt.legend(title='Decoding Type', loc='upper right')
            plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_corss_decoding_performance(all_pred_outcome_corr, brain_regions):

    plt.figure(figsize=(8*len(brain_regions), 8))

    all_pred_outcome_corr = all_pred_outcome_corr[all_pred_outcome_corr['TEMcomponent'].notna()]
    for i, region in enumerate(brain_regions):
        all_pred_outcome_corr_subset = all_pred_outcome_corr[all_pred_outcome_corr['region'] == region]
        plt.subplot(1, len(brain_regions), i+1)
        sns.stripplot(x='label', y='corr', hue='TEMcomponent', data=all_pred_outcome_corr_subset, jitter=True, dodge=True, alpha=0.7, palette='Set2')
        plt.title(f'TEM yhat to Ratings Cross-Decoding Performance\nin {region}')
        plt.xlabel('')
        plt.ylabel('Prediction-Outcome Correlation')
        if i > 0:
            plt.ylabel('')
        plt.legend(title='TEM Component', loc='upper right')
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_subjectConnected_performance_scatter(performance_all, title, legend_title, outer_x_order, outer_x_labels, 
                                              outer_x_column_name, inner_x_order, inner_x_labels, inner_x_column_name, 
                                              y_column_name, inner_x_colors, subject_column_name, outer_shift=1.5, 
                                              inner_shift=0.8, jitter=0.2, figsize=(12, 6), legend_loc='upper left', 
                                              legend_bbox_to_anchor=(1, 1), yrange=None, yaxis_label='Prediction-Outcome Correlation', 
                                              xlabel='', save=None, ax=None, xtick_rotation=None):
    
    inner_x_shift = {group: idx * inner_shift * outer_shift for idx, group in enumerate(inner_x_order)}
    outer_x_shift = {group: idx * len(inner_x_order) * outer_shift for idx, group in enumerate(outer_x_order)}
    
    performance_all['outer_shifted'] = performance_all.apply(
        lambda row: outer_x_shift[row[outer_x_column_name]] + 
                    inner_x_shift[row[inner_x_column_name]] + 
                    np.random.uniform(-jitter, jitter), axis=1
    )

    sns.set(style="white")
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['Arial']
    
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    scatter = sns.scatterplot(data=performance_all, x='outer_shifted', y=y_column_name, hue=inner_x_column_name, 
                              s=100, alpha=0.5, palette=inner_x_colors, ax=ax)

    # subject-wise connection lines
    subjects = performance_all[subject_column_name].unique()
    for subject in subjects:
        subject_data = performance_all[performance_all[subject_column_name] == subject]
        for outer_group in outer_x_order:
            inner_data = subject_data[subject_data[outer_x_column_name] == outer_group]
            inner_data = inner_data.sort_values('outer_shifted')
            if len(inner_data) > 1:
                ax.plot(inner_data['outer_shifted'], inner_data[y_column_name], 
                        color='gray', alpha=0.5, lw=1)
    
    # mean lines for each inner-outer combination
    mean_line_handles = []
    for outer_group in outer_x_order:
        for inner_group in inner_x_order:
            subset = performance_all[(performance_all[outer_x_column_name] == outer_group) & (performance_all[inner_x_column_name] == inner_group)]
            if not subset.empty:
                mean_y = subset[y_column_name].mean()
                mean_x = outer_x_shift[outer_group] + inner_x_shift[inner_group]
                ax.plot([mean_x - jitter, mean_x + jitter], [mean_y, mean_y], 
                        color='black', lw=2, label='_mean_line_')  # Avoid duplicate labels
                mean_line_handles.append(mlines.Line2D([], [], color='black', lw=2, label="Mean"))

    xticks, xticklabels = [], []
    for outer_idx, outer_group in enumerate(outer_x_order):
        xticks.append(outer_x_shift[outer_group] + np.mean([inner_x_shift[i] for i in inner_x_order]))
        xticklabels.append(outer_x_labels[outer_idx])
    
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels, weight='bold')
    ax.set_xlabel(xlabel, weight='bold')
    ax.set_ylabel(yaxis_label, weight='bold')
    ax.set_title(title, weight='bold')

    if yrange:
        ax.set_ylim(yrange)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(False)

    legend_handles = [mlines.Line2D([], [], marker='o', color=color, linestyle='None', markersize=10, label=label, alpha=0.5) 
                      for label, color in zip(inner_x_labels, inner_x_colors)]
    legend_handles.append(mlines.Line2D([], [], color='black', lw=2, label="Mean"))  # Mean line
    ax.legend(handles=legend_handles, title=legend_title, loc=legend_loc, bbox_to_anchor=legend_bbox_to_anchor)

    if xtick_rotation:
        plt.xticks(rotation=xtick_rotation)
    plt.tight_layout()

    if save:
        plt.savefig(save, dpi=600)

    if ax is None:
        plt.show()

In [ ]:
va_dir = './outputs/rep3/ratings_prediction_yhat/brain/valence_arousal'
category_dir = './outputs/rep3/ratings_prediction_yhat/brain//category'
ori_category_dir = './outputs/rep3/ratings_prediction_performance/brain/category/categoryRatings_prediction_performance_generalized_across_movies_hcecvmpfc.csv'
ori_va_dir = './outputs/rep3/ratings_prediction_performance/brain/valence_arousal/valenceArousalRatings_prediction_performance_generalized_across_movies_hcecvmpfc.csv'
brain_regions = ['Hippocampus', 'EntorhinalCortex', 'vmPFC']#['anteriorHippocampus', 'posteriorHippocampus']
components = ['p','g']
categories = ['Anger', 'Anxiety', 'Fear', 'Surprise', 'Guilt', 'Disgust', 'Sad', 'Regard', 
            'Satisfaction', 'WarmHeartedness', 'Happiness', 'Pride', 'Love']
va_ratings = ['Good', 'Bad', 'Calm', 'AtEase']
TEM_dir = './outputs/rep3/ratings_prediction_yhat/brainToTEM/MDSseed121/iteration32000/walkRandomSeed42'
# Get subject list
subjects = list(set([
fname.split('/')[-1].split('_')[1].split('-')[1]
for fname in glob.glob(os.path.join(va_dir, 'yhat_sub-S*.csv'))
]))
#partial_corr_df = partial_correlation_BrainTEMRatings(TEM_dir, va_dir, category_dir, subjects, categories, va_ratings)
partial_corr_df = partial_correlation_BrainTEMRatings(TEM_dir, va_dir, category_dir, subjects, categories, va_ratings,
                                                      emo_types=['category', 'valence_arousal'], 
                                                      brain_regions=['Hippocampus','Hippocampus'], 
                                                      components=['p','p'])



In [ ]:
partial_corr_df_melted = pd.melt(partial_corr_df, id_vars=['subject', 'region', 'TEM_component', 'emo_type','Emotion','freq'], value_vars=['r_xy_z', 'r_xy_given_m_z'], var_name='type', value_name='zvalue')
partial_corr_df_melted['corr_type'] = partial_corr_df_melted['type'].replace({'r_xy_given_m_z':'partial_corr', 'r_xy_z':'corr'})
partial_corr_df_melted = partial_corr_df_melted.drop(columns='type')
ori_df = pd.read_csv('./outputs/performance_partial_corr_brainToTEMtoRatings_apHC_it32000.csv')
partial_corr_df_melted = pd.concat([ori_df, partial_corr_df_melted], ignore_index=True)
partial_corr_df_melted.to_csv('./outputs/performance_partial_corr_brainToTEMtoRatings_it32000.csv', index=False)

In [11]:
partial_corr_df = pd.read_csv('./outputs/performance_partial_corr_brainToTEMtoRatings_it32000.csv')
partial_corr_df = partial_corr_df[partial_corr_df['TEM_component'] == 'p']
partial_corr_df['corr'] = reverse_fisher_z(partial_corr_df['zvalue'])
partial_corr_df = partial_corr_df[partial_corr_df['region'] == 'Hippocampus']

partial_corr_df = partial_corr_df.pivot_table(index=['subject', 'region', 'TEM_component', 'emo_type','Emotion'], columns='corr_type', values='corr').reset_index()
#get partial_corr - corr difference for each row
partial_corr_df['partial_corr_diff'] = partial_corr_df['partial_corr'] - partial_corr_df['corr']
#get average across partial_corr_diff across Emotion and subject to get one single value for each emo_type
print('mean diff')
print(partial_corr_df.groupby(['emo_type'])['partial_corr_diff'].mean())


def bootstrap_ci(data, statistic=np.mean, n_bootstrap=1000, ci=0.95):
    data = np.asarray(data)

    bootstrap_samples = np.random.choice(data, (n_bootstrap, len(data)), replace=True)
    bootstrap_statistics = np.apply_along_axis(statistic, 1, bootstrap_samples)
    lower_percentile = (1 - ci) / 2
    upper_percentile = 1 - lower_percentile
    
    ci_lower = np.percentile(bootstrap_statistics, lower_percentile * 100)
    ci_upper = np.percentile(bootstrap_statistics, upper_percentile * 100)
    
    return (ci_lower, ci_upper)

def bootstrap_analysis(grouped_data):
    results = []
    
    for emo_type in ['category','valence_arousal']:
        group = grouped_data[emo_type]

        ci = bootstrap_ci(group)
        
        results.append({
            'emo_type': emo_type,
            'Mean': group.mean(),
            'CI_Lower': ci[0],
            'CI_Upper': ci[1]
        })
    
    return pd.DataFrame(results)

results = bootstrap_analysis(partial_corr_df.groupby(['emo_type','subject'])['partial_corr_diff'].mean())
print(results)

mean diff
emo_type
category          -0.004047
valence_arousal   -0.000214
Name: partial_corr_diff, dtype: float64
          emo_type      Mean  CI_Lower  CI_Upper
0         category -0.004047 -0.006273 -0.001961
1  valence_arousal -0.000214 -0.001421  0.001004
